# Chapter 08-03: TensorFlow Lite 변환과 양자화

## 학습 목표
- TFLite로 모델을 변환하여 모바일/엣지 디바이스에 배포하는 흐름을 이해한다
- Post-Training Quantization의 세 가지 방식(Dynamic, Float16, Int8)을 구분한다
- TFLite 인터프리터로 직접 추론을 실행한다
- 모델 크기와 추론 속도를 비교·분석한다

## 목차
1. 양자화 수학적 원리
2. MNIST 모델 학습 및 저장
3. TFLite 기본 변환
4. Dynamic Range Quantization
5. Float16 Quantization
6. TFLite 인터프리터 추론
7. 모델 크기 비교
8. 정리

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import os
import time

print(f"TensorFlow 버전: {tf.__version__}")

# MNIST 데이터 로드 및 전처리
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0
X_train = X_train[..., np.newaxis]  # (60000, 28, 28, 1)
X_test  = X_test[..., np.newaxis]   # (10000, 28, 28, 1)

# TFLite 파일 저장 경로
TFLITE_DIR = '/tmp/tflite_models'
os.makedirs(TFLITE_DIR, exist_ok=True)

print(f"학습 데이터: {X_train.shape}")
print(f"TFLite 저장 경로: {TFLITE_DIR}")

## 1. 양자화(Quantization) 수학적 원리

양자화는 부동소수점(float32)을 정수(int8/int16)로 압축하여 모델 크기와 연산 비용을 줄인다.

### 선형 양자화 공식

**양자화 (float → int)**

$$x_{int} = \text{round}\left(\frac{x_{float}}{s}\right) + z$$

**역양자화 (int → float)**

$$x_{float} \approx s\,(x_{int} - z)$$

- $s$ (scale): 부동소수점 범위를 정수 범위로 매핑하는 스케일 계수
- $z$ (zero-point): 0.0 을 표현하는 정수 값 (비대칭 양자화)

### 방식별 비교

| 방식 | 가중치 | 활성화 | 크기 감소 | 속도 향상 | 정확도 손실 |
|------|--------|--------|-----------|-----------|-------------|
| Float32 (원본) | float32 | float32 | 기준 | 기준 | 없음 |
| Dynamic Range | int8 | float32 (추론 시) | ~75% | ~2–3x | 매우 작음 |
| Float16 | float16 | float16 | ~50% | GPU에서 향상 | 매우 작음 |
| Full Int8 | int8 | int8 | ~75% | ~3–4x | 작음 |

## 2. 간단한 MNIST 모델 학습 후 저장

In [ ]:
# MNIST 분류 CNN 모델 구성
def build_mnist_model():
    """TFLite 변환 실습용 MNIST CNN 모델"""
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ], name='mnist_cnn')
    return model

model = build_mnist_model()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# 빠른 학습 (에포크 3회)
model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=128,
    verbose=1
)

# 테스트 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n원본 모델 테스트 정확도: {test_acc:.4f}")

# .keras 형식으로 저장
keras_path = os.path.join(TFLITE_DIR, 'mnist_original.keras')
model.save(keras_path)
print(f"원본 모델 저장: {keras_path}")

## 3. TFLite 기본 변환 (양자화 없음)

`TFLiteConverter`는 Keras 모델, SavedModel 등 다양한 소스에서 TFLite 파일을 생성한다.

In [ ]:
# Keras 모델에서 TFLite 컨버터 생성
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 기본 변환 (양자화 없이 float32 유지)
tflite_model_fp32 = converter.convert()

# .tflite 파일로 저장
path_fp32 = os.path.join(TFLITE_DIR, 'mnist_fp32.tflite')
with open(path_fp32, 'wb') as f:
    f.write(tflite_model_fp32)

size_fp32 = os.path.getsize(path_fp32)
print(f"FP32 TFLite 모델 크기: {size_fp32:,} bytes ({size_fp32/1024:.1f} KB)")

## 4. Post-Training Dynamic Range Quantization

가중치를 int8로 양자화하고, 활성화는 추론 시점에 동적으로 양자화한다.
별도의 보정 데이터 없이 적용 가능하므로 가장 간단한 방법이다.

In [ ]:
# Dynamic Range Quantization 설정
converter_dynamic = tf.lite.TFLiteConverter.from_keras_model(model)

# 최적화 플래그: DEFAULT → Dynamic Range Quantization 활성화
converter_dynamic.optimizations = [tf.lite.Optimize.DEFAULT]

# 변환 실행
tflite_model_dynamic = converter_dynamic.convert()

# 저장
path_dynamic = os.path.join(TFLITE_DIR, 'mnist_dynamic_quant.tflite')
with open(path_dynamic, 'wb') as f:
    f.write(tflite_model_dynamic)

size_dynamic = os.path.getsize(path_dynamic)
print(f"Dynamic Quant 모델 크기: {size_dynamic:,} bytes ({size_dynamic/1024:.1f} KB)")
print(f"크기 감소율: {(1 - size_dynamic/size_fp32)*100:.1f}%")

## 5. Post-Training Float16 Quantization

가중치와 연산을 float16으로 변환한다.
GPU를 탑재한 모바일 기기(예: iPhone Neural Engine)에서 성능 이점이 크다.

In [ ]:
# Float16 Quantization 설정
converter_fp16 = tf.lite.TFLiteConverter.from_keras_model(model)
converter_fp16.optimizations = [tf.lite.Optimize.DEFAULT]

# 지원 타입을 float16으로 제한
converter_fp16.target_spec.supported_types = [tf.float16]

# 변환 실행
tflite_model_fp16 = converter_fp16.convert()

# 저장
path_fp16 = os.path.join(TFLITE_DIR, 'mnist_fp16_quant.tflite')
with open(path_fp16, 'wb') as f:
    f.write(tflite_model_fp16)

size_fp16 = os.path.getsize(path_fp16)
print(f"Float16 Quant 모델 크기: {size_fp16:,} bytes ({size_fp16/1024:.1f} KB)")
print(f"크기 감소율 (vs FP32): {(1 - size_fp16/size_fp32)*100:.1f}%")

## 6. TFLite 인터프리터로 추론

`tf.lite.Interpreter`를 사용하면 변환된 `.tflite` 파일을 로드하고 추론을 실행할 수 있다.
모바일 기기에서의 동작을 PC에서 시뮬레이션하는 용도로도 활용한다.

In [ ]:
def run_tflite_inference(tflite_path, test_images, test_labels, num_samples=500):
    """
    TFLite 인터프리터로 추론을 실행하고 정확도와 평균 추론 시간을 반환한다.

    Args:
        tflite_path: .tflite 파일 경로
        test_images: 테스트 이미지 배열
        test_labels: 정답 레이블 배열
        num_samples: 평가할 샘플 수
    Returns:
        accuracy (float), avg_time_ms (float)
    """
    # 인터프리터 초기화
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()  # 텐서 메모리 할당

    # 입출력 텐서 정보 조회
    input_details  = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    correct = 0
    total_time = 0.0

    for i in range(num_samples):
        # 단일 이미지 배치 형태로 준비 [1, 28, 28, 1]
        input_data = np.expand_dims(test_images[i], axis=0)

        # 입력 텐서에 데이터 설정
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # 추론 실행 + 시간 측정
        start = time.perf_counter()
        interpreter.invoke()
        total_time += (time.perf_counter() - start) * 1000  # ms 단위

        # 출력 텐서에서 결과 가져오기
        output_data = interpreter.get_tensor(output_details[0]['index'])
        pred = np.argmax(output_data[0])

        if pred == test_labels[i]:
            correct += 1

    accuracy = correct / num_samples
    avg_time = total_time / num_samples
    return accuracy, avg_time

# 입출력 텐서 정보 출력
interp = tf.lite.Interpreter(model_path=path_fp32)
interp.allocate_tensors()
print("입력 텐서 정보:")
for d in interp.get_input_details():
    print(f"  name={d['name']}, shape={d['shape']}, dtype={d['dtype']}")
print("출력 텐서 정보:")
for d in interp.get_output_details():
    print(f"  name={d['name']}, shape={d['shape']}, dtype={d['dtype']}")

print("\n추론 실행 중... (각 500개 샘플)")
acc_fp32,    time_fp32    = run_tflite_inference(path_fp32,    X_test, y_test)
acc_dynamic, time_dynamic = run_tflite_inference(path_dynamic, X_test, y_test)
acc_fp16,    time_fp16    = run_tflite_inference(path_fp16,    X_test, y_test)

print(f"\nFP32    - 정확도: {acc_fp32:.4f}, 평균 추론 시간: {time_fp32:.3f} ms")
print(f"Dynamic - 정확도: {acc_dynamic:.4f}, 평균 추론 시간: {time_dynamic:.3f} ms")
print(f"FP16    - 정확도: {acc_fp16:.4f}, 평균 추론 시간: {time_fp16:.3f} ms")

## 7. 모델 크기 비교

In [ ]:
# 원본 Keras 모델 크기 포함 전체 비교
original_size = os.path.getsize(keras_path)

print("=" * 55)
print(f"{'형식':<22} {'크기(KB)':>10} {'감소율':>10} {'정확도':>10}")
print("-" * 55)
print(f"{'원본 .keras':<22} {original_size/1024:>10.1f} {'기준':>10} {test_acc:>10.4f}")
print(f"{'TFLite FP32':<22} {size_fp32/1024:>10.1f} {0.0:>9.1f}% {acc_fp32:>10.4f}")
print(f"{'TFLite Dynamic Quant':<22} {size_dynamic/1024:>10.1f} {(1-size_dynamic/size_fp32)*100:>9.1f}% {acc_dynamic:>10.4f}")
print(f"{'TFLite FP16 Quant':<22} {size_fp16/1024:>10.1f} {(1-size_fp16/size_fp32)*100:>9.1f}% {acc_fp16:>10.4f}")
print("=" * 55)

# 파일 목록 확인
print("\n생성된 파일 목록:")
for fname in sorted(os.listdir(TFLITE_DIR)):
    fpath = os.path.join(TFLITE_DIR, fname)
    print(f"  {fname:<35} {os.path.getsize(fpath)/1024:>8.1f} KB")

## 8. 정리 - 양자화 방식 비교

### 양자화 방식 선택 기준

| 양자화 방식 | 보정 데이터 필요 | 크기 감소 | 추천 대상 |
|-------------|-----------------|-----------|----------|
| 없음 (FP32) | 불필요 | 없음 | 변환 검증, 디버깅 |
| Dynamic Range | 불필요 | ~75% | CPU 엣지 디바이스, 빠른 배포 |
| Float16 | 불필요 | ~50% | GPU/NPU 탑재 디바이스 |
| Full Int8 | **필요** (대표 데이터 100~500개) | ~75% | 최고 성능이 필요한 경우 |

### TFLite 변환 핵심 API

```python
# 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]   # 양자화 활성화
tflite_model = converter.convert()

# 추론
interp = tf.lite.Interpreter(model_path='model.tflite')
interp.allocate_tensors()
interp.set_tensor(input_idx, input_data)
interp.invoke()
output = interp.get_tensor(output_idx)
```